### Data Loading 

In [50]:
import pandas as pd

users = pd.read_csv(r"D:/WIZCOMMERCE/data-analytics-assignment/Data/users_v2 (1).csv")
events = pd.read_csv(r"D:/WIZCOMMERCE/data-analytics-assignment/Data/events_v2 (1).csv")
payments = pd.read_csv(r"D:/WIZCOMMERCE/data-analytics-assignment/Data/payments_v2 (1).csv")

### Data Preprocessing 

In [51]:
users['signup_date'] = pd.to_datetime(users['signup_date'])
events['event_time'] = pd.to_datetime(events['event_time'])
payments['payment_date'] = pd.to_datetime(payments['payment_date'])

### Missing Values Check 

In [52]:
print("Missing Values :-\n")

print(users.isna().sum(), "\n")
print(events.isna().sum(), "\n")
print(payments.isna().sum(), "\n")

Missing Values :-

user_id        0
signup_date    0
country        0
device         0
source         0
dtype: int64 

user_id       0
event_name    0
event_time    0
dtype: int64 

user_id         0
plan_type       0
amount          0
payment_date    0
dtype: int64 



### Funnel : 1 — Signups 

In [53]:
unique_signed_up_users = users['user_id'].nunique()
print("Signed Up Users:", unique_signed_up_users)

Signed Up Users: 800


### Funnel : 2 — Viewed Feature  

In [54]:
viewed_feature = events[events['event_name'] == "viewed_feature"]
unique_viewed_feature_users = viewed_feature['user_id'].nunique()
print("Viewed Feature:", unique_viewed_feature_users)

Viewed Feature: 702


### Funnel : 3 — Returned Within 7 Days 

In [55]:
merged = events.merge(users, on="user_id")
merged['days_from_signup'] = (merged['event_time'] - merged['signup_date']).dt.days
returned = merged[merged['days_from_signup'].between(0, 7)]
unique_users_returned_7d = returned['user_id'].nunique()
print("Returned in 7 Days:", unique_users_returned_7d)

Returned in 7 Days: 800


### Funnel : 4 — Upgraded 

In [56]:
upgraded_users_count = payments['user_id'].nunique()
print("Upgraded:", upgraded_users_count)

Upgraded: 224


### Conversion Rates 

In [58]:
conv_signup_to_feature = unique_viewed_feature_users / unique_signed_up_users
conv_feature_to_return = (
    unique_users_returned_7d / unique_viewed_feature_users
    if unique_viewed_feature_users
    else 0
)
conv_return_to_upgrade = (
    upgraded_users_count / unique_users_returned_7d
    if unique_users_returned_7d
    else 0
)
overall_conversion_rate = upgraded_users_count / unique_signed_up_users

print("\nConversion Rates :-")
print("Signup → Feature :", round(conv_signup_to_feature, 4))
print("Feature → Return :", round(conv_feature_to_return, 4))
print("Return → Upgrade :", round(conv_return_to_upgrade, 4))
print("Overall Conversion :", round(overall_conversion_rate, 4))


Conversion Rates :-
Signup → Feature : 0.8775
Feature → Return : 1.1396
Return → Upgrade : 0.28
Overall Conversion : 0.28


### Weekly Retention Cohort 

In [39]:
merged['week_since_signup'] = (
    (merged['event_time'] - merged['signup_date']).dt.days // 7
)

weekly_retention_counts = merged.groupby('week_since_signup')['user_id'].nunique()
print("\nWeekly Retention :-\n", weekly_retention_counts)


Weekly Retention :-
 week_since_signup
0    800
1    431
2    457
3    428
4    441
5    408
Name: user_id, dtype: int64


### Upgrade within 30 Days 

In [47]:
payment_data = payments.merge(users, on="user_id")
payment_data['days_to_upgrade'] = (
    payment_data['payment_date'] - payment_data['signup_date']
).dt.days

upgraded_within_30d = payment_data[payment_data['days_to_upgrade'] <= 30]['user_id'].nunique()
upgrade_rate_30d = upgraded_within_30d / unique_signed_up_users

print("\nUpgrade Rate of 30-Day :", round(upgrade_rate_30d, 4))
print("Upgrades Within 30 Days Interval :", upgraded_within_30d)


Upgrade Rate of 30-Day : 0.28
Upgrades Within 30 Days Interval : 224


### Segmentation Analysis 

In [35]:
def segment_funnel(df, segment_col):
    segment_stats = []

    for seg_value, seg_group in df.groupby(segment_col):
        seg_user_ids = seg_group['user_id'].unique()

        n_signed_up = len(seg_user_ids)
        n_viewed_feature = viewed_feature[
            viewed_feature['user_id'].isin(seg_user_ids)
        ]['user_id'].nunique()
        n_returned_7d = returned[
            returned['user_id'].isin(seg_user_ids)
        ]['user_id'].nunique()
        n_upgraded = payments[
            payments['user_id'].isin(seg_user_ids)
        ]['user_id'].nunique()

        segment_stats.append({
            segment_col: seg_value,
            "num_signed_up": n_signed_up,
            "num_viewed_feature": n_viewed_feature,
            "num_returned_7d": n_returned_7d,
            "num_upgraded": n_upgraded,
            "paid_conversion_rate": round(n_upgraded / n_signed_up, 4)
            if n_signed_up
            else 0
        })

    return pd.DataFrame(segment_stats)


country_funnel_stats = segment_funnel(users, "country")
source_funnel_stats = segment_funnel(users, "source")
device_funnel_stats = segment_funnel(users, "device")

print("\nCountry Segment Funnel:\n", country_funnel_stats)
print("\nSource Segment Funnel:\n", source_funnel_stats)
print("\nDevice Segment Funnel:\n", device_funnel_stats)



Country Segment Funnel:
      country  num_signed_up  num_viewed_feature  num_returned_7d  \
0  Australia            107                  92              107   
1     Brazil            105                  92              105   
2     Canada            128                 107              128   
3    Germany            121                 104              121   
4      India            113                 101              113   
5         UK            108                  96              108   
6        USA            118                 110              118   

   num_upgraded  paid_conversion_rate  
0            37                0.3458  
1            21                0.2000  
2            38                0.2969  
3            32                0.2645  
4            40                0.3540  
5            26                0.2407  
6            30                0.2542  

Source Segment Funnel:
      source  num_signed_up  num_viewed_feature  num_returned_7d  num_upgraded  \
0  

### Conversion Signals 

In [34]:
user_total_events = events.groupby('user_id')['event_name'].count()
user_distinct_events = events.groupby('user_id')['event_name'].nunique()
user_active_days = events.groupby('user_id')['event_time'].apply(
    lambda x: x.dt.date.nunique()
)

user_behavior_stats = pd.DataFrame({
    "total_events": user_total_events,
    "num_distinct_events": user_distinct_events,
    "num_active_days": user_active_days
}).reset_index()

user_behavior_stats['is_upgraded'] = user_behavior_stats['user_id'].isin(
    payments['user_id']
)

print("\nConversion Signal Summary :-\n")
print(user_behavior_stats.groupby("is_upgraded").mean(numeric_only=True))


Conversion Signal Summary :-

                user_id  total_events  num_distinct_events  num_active_days
is_upgraded                                                                
False        398.479167      4.605903             3.251736         4.411458
True         405.696429      9.102679             4.866071         8.281250
